In [15]:
# Recreate the full script with original inputs and updated output formatting

import numpy as np
import pandas as pd
import math

# ----------------------------
# MERGED INPUT SECTION
# ----------------------------

months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]

# Input data from user
actual_kwh = [3500, 6000, 2000, 1000, 0, 100, 930, 1500, 5000, 5500, 6500, 8000]
days_per_month = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
climate_index_h = [1.01, 0.93, 0.83, 0.79, 0.65, 0.91, 0.64, 0.72, 0.97, 1.02, 1.00, 1.02]

# Technical constants
motor_kw = 11
solar_kw = 15.54
drive_min_pct = 0.35
loss_factor_hourly = 0.0
loss_factor_monthly = 0.20
kwh_cost = 2.6

# Solar production profile (Veichi Tech!B33:B45)
sunshine_percent = np.array([0, 0.03, 0.32, 0.5, 0.75, 0.9, 1.0, 0.99, 0.85, 0.65, 0.3, 0.01, 0])

# ----------------------------
# CALCULATION BLOCK 1: Pump Hours Per Day
# ----------------------------

kwh_per_day = [actual_kwh[i] / days_per_month[i] for i in range(12)]
monthly_pump_hours = {months[i]: round(kwh_per_day[i] / motor_kw, 9) for i in range(12)}

# ----------------------------
# CALCULATION BLOCK 2: Hourly Yield (Veichi Hourly Block)
# ----------------------------

sizing_factor = solar_kw / motor_kw
drive_output = np.minimum(sunshine_percent * sizing_factor, 1.0)
F = np.full_like(drive_output, motor_kw * 1000)
G = np.where(drive_output >= drive_min_pct, drive_output * F, 0)
H = np.where(G > 0, G * (1 - loss_factor_hourly), 0)
I = H / F

def build_excel_matched_distribution_fixed(total_hours):
    spread = np.zeros(13)
    center = 6
    if total_hours <= 1:
        spread[center] = total_hours
        return spread
    spread[center] = 1
    used = 1
    offset = 1
    while used + 2 <= total_hours and offset <= 6:
        spread[center - offset] = 1
        spread[center + offset] = 1
        used += 2
        offset += 1
    remaining = total_hours - used
    if remaining > 0 and offset <= 6:
        spread[center - offset] = remaining / 2
        spread[center + offset] = remaining / 2
    return spread

# Hourly Yield Calculation
hourly_yield_j = []
for month in months:
    spread = build_excel_matched_distribution_fixed(monthly_pump_hours[month])
    output_kwh = float(np.sum(spread * I))
    hourly_yield_j.append(round(output_kwh, 9))

# ----------------------------
# CALCULATION BLOCK 3: Final Monthly and Yearly Savings
# ----------------------------

results = []
for i in range(12):
    month = months[i]
    actual = actual_kwh[i]
    days = days_per_month[i]
    j = hourly_yield_j[i]
    h = climate_index_h[i]
    k = round(j * h, 9)
    expenditure = round(actual * kwh_cost, 2)
    kwh_day = actual / days
    pump_hours_day = kwh_day / motor_kw
    theoretical_savings = round(k * kwh_cost * motor_kw * days, 10)
    capped_savings = round(min(theoretical_savings, expenditure), 10)
    actual_savings = round(capped_savings * (1 - loss_factor_monthly), 10)

    results.append({
        "Month": month,
        "Pump Hours/Day": round(pump_hours_day, 9),
        "Hourly Yield (J)": j,
        "Yield With Climate (K)": k,
        "Expenditure (R)": expenditure,
        "Theoretical Savings": theoretical_savings,
        "Capped Savings": capped_savings,
        "Actual Savings (L)": actual_savings
    })

final_df = pd.DataFrame(results)
total_savings = round(sum(row["Actual Savings (L)"] for row in results), 2)

# ------------------------------
# MERGED PAYBACK BLOCK
# ------------------------------

# Financial assumptions
tax_rate = 0.27
system_cost = 180000
first_year_savings = total_savings
deposit = 0.10
loan_period_years = 10
loan_period_months = loan_period_years * 12
interest_rate_monthly = 0.12 / 12
elec_increase_5y = 0.14
elec_increase_6y = 0.06


# Loan Calculations
loan_amount = system_cost * (1 - deposit)
monthly_loan_repayment = (loan_amount * interest_rate_monthly) * ((1 + interest_rate_monthly) ** loan_period_months) / (((1 + interest_rate_monthly) ** loan_period_months) - 1)
annual_loan_repayment = monthly_loan_repayment * 12
total_financed_cost = -(annual_loan_repayment * loan_period_years + (system_cost * deposit))

# Loan Table
df = pd.DataFrame(index=range(1, 16))
df["Year"] = ["Year " + str(i) for i in df.index]
df["Loan Repayment"] = [-annual_loan_repayment if i <= loan_period_years else 0 for i in df.index]
df.at[1, "Loan Repayment"] -= system_cost * deposit

# Electricity Savings
savings = first_year_savings
df["Electricity Savings"] = 0.0
df.at[1, "Electricity Savings"] = savings
for i in range(2, 16):
    if i <= 5:
        savings *= (1 + elec_increase_5y)
    else:
        savings *= (1 + elec_increase_6y)
    df.at[i, "Electricity Savings"] = savings

# Tax and Total Savings
df["Tax Savings"] = [system_cost * tax_rate if i == 1 else 0 for i in df.index]
df["Electricity + Tax Savings"] = df["Electricity Savings"] + df["Tax Savings"]
df["Cashflow"] = df["Loan Repayment"] + df["Electricity + Tax Savings"]
df["Cumulative Cashflow"] = df["Cashflow"].cumsum()
df["Cost Per kWh"] = -df["Loan Repayment"] / first_year_savings_kWh
df["Cum Break Even"] = df["Electricity + Tax Savings"].cumsum() + total_financed_cost

# Break-even (Loan) - Linear Interpolation
if (df["Cum Break Even"] > 0).any():
    idx = df[df["Cum Break Even"] > 0].index[0]
    if idx > 1:
        val_before = df.at[idx - 1, "Cum Break Even"]
        val_after = df.at[idx, "Cum Break Even"]
        fraction = (0 - val_before) / (val_after - val_before)
        break_even_year = idx - 1
        break_even_month = round(fraction * 12)
    else:
        break_even_year = 0
        break_even_month = round((0 - 0) / df.at[idx, "Cum Break Even"] * 12)
else:
    break_even_year = break_even_month = None

# Cash Break-even - Linear Interpolation
def calculate_cash_payback_interpolated(savings_list, system_cost):
    cumulative = 0
    for i, yearly_savings in enumerate(savings_list, start=1):
        previous = cumulative
        cumulative += yearly_savings
        if cumulative >= system_cost:
            excess = cumulative - system_cost
            fraction = (yearly_savings - excess) / yearly_savings
            return i - 1, round(fraction * 12)
    return None, None

cash_savings_list = df["Electricity + Tax Savings"].tolist()
cash_payback_year, cash_payback_month = calculate_cash_payback_interpolated(cash_savings_list, system_cost)

# Cash Table
df_cash = pd.DataFrame(index=range(1, 16))
df_cash["Year"] = ["Year " + str(i) for i in df_cash.index]
df_cash["Electricity + Tax Savings"] = df["Electricity + Tax Savings"]
df_cash["Cumulative Savings"] = df_cash["Electricity + Tax Savings"].cumsum()
df_cash["System Cost"] = system_cost
df_cash["Net Position"] = df_cash["Cumulative Savings"] - system_cost

# LCOE
kWh_by_year = []
kWh = first_year_savings_kWh
for i in range(1, 16):
    kWh_by_year.append(kWh)
    if i < 5:
        kWh *= (1 + elec_increase_5y)
    else:
        kWh *= (1 + elec_increase_6y)
total_kWh_15y = sum(kWh_by_year)
cash_LCOE = system_cost / total_kWh_15y
loan_LCOE = -df["Loan Repayment"].sum() / total_kWh_15y

# Format output layout
from io import StringIO
output = StringIO()
output.write("Loan Amount: R{:,.2f}\n".format(loan_amount))
output.write("Monthly Loan Repayment: R{:,.2f}\n".format(monthly_loan_repayment))
output.write("Annual Loan Repayment: R{:,.2f}\n".format(annual_loan_repayment))
output.write("Total Financed Cost: R{:+,.2f}\n".format(total_financed_cost))
output.write("\n🔹 Cost per kWh (15-Year Average):\n")
output.write("Cash Purchase: R{:.2f} per kWh\n".format(cash_LCOE))
output.write("Financed Purchase: R{:.2f} per kWh\n".format(loan_LCOE))
output.write("\nBreak Even (Cash): ")
output.write(f"{cash_payback_year} Years and {cash_payback_month} Months\n" if cash_payback_year is not None else "No break-even (Cash) within 15 years.\n")
output.write("Break Even (Loan): ")
output.write(f"{break_even_year} Years and {break_even_month} Months\n" if break_even_year is not None else "No break-even (Loan) within 15 years.\n")
output.write("\n--- Loan-Based Financial Model ---\n\n")
output.write(df.to_string(index=True))
output.write("\n\n--- Cash-Based Payback Table ---\n\n")
output.write(df_cash.to_string(index=True))
formatted_output = output.getvalue()
print(formatted_output)


Loan Amount: R162,000.00
Monthly Loan Repayment: R2,324.23
Annual Loan Repayment: R27,890.75
Total Financed Cost: R-296,907.52

🔹 Cost per kWh (15-Year Average):
Cash Purchase: R0.40 per kWh
Financed Purchase: R0.66 per kWh

Break Even (Cash): 2 Years and 11 Months
Break Even (Loan): 4 Years and 10 Months

--- Loan-Based Financial Model ---

       Year  Loan Repayment  Electricity Savings  Tax Savings  Electricity + Tax Savings       Cashflow  Cumulative Cashflow  Cost Per kWh  Cum Break Even
1    Year 1   -45890.752369         39511.080000      48600.0               88111.080000   42220.327631         42220.327631      3.059383  -208796.443695
2    Year 2   -27890.752369         45042.631200          0.0               45042.631200   17151.878831         59372.206461      1.859383  -163753.812495
3    Year 3   -27890.752369         51348.599568          0.0               51348.599568   23457.847199         82830.053660      1.859383  -112405.212927
4    Year 4   -27890.752369         